### Heroes Of Pymoli Data Analysis
* Of the 1163 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  
-----

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [57]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
df = pd.read_csv(file_to_load)
df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

* Display the total number of players


In [58]:
# The unique method shows every element of the series that appears only once
unique = df["SN"].unique()
unique_len = len(df["SN"].unique())
total_players=pd.DataFrame({"Total Players":[unique_len]})
total_players

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [59]:
# The unique method shows every element of the series that appears only once
# Did not work when used with Item Name, presumably because games with the same name had multiple Item IDs
unique_items = len(df["Item ID"].unique())

# Avg price, mean of the specific column
avg_price=df.loc[:,"Price"].mean()
formatted_avg_price='${0:.2f}'.format(avg_price)
formatted_avg_price_string = str(formatted_avg_price)

# Total Number of Purchases
total_purchases=df["Purchase ID"].count()
total_purchases_string=str(total_purchases) 

# Total Revenue, The sum method adds every entry in the series
total_rev = df["Price"].sum()
formatted_total_rev = '${0:.2f}'.format(total_rev)
formatted_total_rev_string= str(formatted_total_rev)

# Place all of the data found into a summary DataFrame
summary_table = pd.DataFrame({"Unique Items": unique_items,
                              "Average Price": formatted_avg_price,
                              "Total Purchases": total_purchases,
                              "Total Revenue": formatted_total_rev}, index=[0])


# Difference between 2 summary tables -> 1st one does not have square brackets around variables, but has index=[0] after curly bracket
summary_table_method2 = pd.DataFrame({"Unique Items": [unique_items],
                              "Average Price": [formatted_avg_price],
                              "Total Purchases": [total_purchases],
                              "Total Revenue": [formatted_total_rev]})

summary_table

,Unique Items,Average Price,Total Purchases,Total Revenue
0,183,$3.05,780,$2379.77


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [60]:
# The value_counts method counts unique values in a column

#purch_count_by_gen=df.groupby('Gender')['SN'].count().reset_index()
purch_count_by_gen=df[['Gender','SN']].copy()
purch_count_by_gen.drop_duplicates(['SN'],keep='first', inplace=True)
purch_count_by_gen_2=purch_count_by_gen.groupby('Gender')['SN'].count().reset_index()
#purch_count_by_gen_2.set_index('Gender', inplace=True)
gen_counts=purch_count_by_gen_2[['Gender','SN']]
gen_counts.rename(columns={gen_counts.columns[1]: "User_Count"}, inplace=True)
gen_counts.set_index('Gender',inplace=True)
unique_int=int(len(unique))


# Use .loc function to get values for the gender counts
male_count=gen_counts.loc["Male","User_Count"]
female_count=gen_counts.loc["Female","User_Count"]
other_count=gen_counts.loc["Other / Non-Disclosed","User_Count"]

# Calculate & Format Percentages
male_per=((male_count/unique_int)*100)
female_per=((female_count/unique_int)*100)
other_per=((other_count/unique_int)*100)



# Create Data Table for Percentages
gender_per=pd.DataFrame({"Male": [male_per], "Female": [female_per], "Other / Non-Disclosed": [other_per]}).transpose()
#Rename Index
gender_per.index.name = ('Gender')
#Rename First Column
gender_per.rename(columns={gender_per.columns[0]: "Gender Percentage"}, inplace=True)

# Merge two dataframes using an inner join
# inner join will only return rows with a key that is present in both tables
gen_merge = pd.merge(gen_counts, gender_per, on="Gender")
gen_merge['Gender Percentage'] = gen_merge['Gender Percentage'].map("{:,.2f}%".format)
gen_merge=gen_merge.sort_values(['User_Count'], ascending=[False])


gen_merge

,User_Count,Gender Percentage
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [61]:
# Set Index by Gender
df.set_index('Gender')

# Total Purchase Value
purch_val_by_gen=df.groupby('Gender')['Price'].sum().reset_index()
purch_val_by_gen.rename(columns={purch_val_by_gen.columns[1]: "Total_Purchase_Value"}, inplace=True)
#purch_val_by_gen= int(purch_val_by_gen)

# Total Purchase Count
purch_count_by_gen=df.groupby('Gender')['SN'].count().reset_index()
purch_count_by_gen.rename(columns={purch_count_by_gen.columns[1]: "Purchase_Count"}, inplace=True)



gen_merge_2 = pd.merge(purch_val_by_gen, purch_count_by_gen, on="Gender")
gen_merge_2['Avg_Purchase_Price']=(gen_merge_2['Total_Purchase_Value'])/(gen_merge_2['Purchase_Count'])
gen_merge_2['Avg_Purchase_Price']=gen_merge_2.Avg_Purchase_Price.astype(str)
gen_merge_2['Avg_Purchase_Price']=(gen_merge_2['Total_Purchase_Value'])/(gen_merge_2['Purchase_Count'])
gen_merge_2['Avg_Purchase_Price']=gen_merge_2['Avg_Purchase_Price'].map("${:,.2f}".format)
gen_merge_3=pd.merge(gen_merge,gen_merge_2, on="Gender")
gen_merge_3['Total_Purchase_Value'].astype(float)
avg_tot_per_person_by_gen=(gen_merge_3['Total_Purchase_Value'])/(gen_merge_3['User_Count']).astype(float)
gen_merge_3['Avg Tot per Person by Gen']=avg_tot_per_person_by_gen
gen_merge_3['Avg Tot per Person by Gen']=gen_merge_3['Avg Tot per Person by Gen'].map("${:,.2f}".format)
gen_merge_3['Total_Purchase_Value']=gen_merge_3['Total_Purchase_Value'].map("${:,.2f}".format)
gen_merge_3 = gen_merge_3.drop(['User_Count','Gender Percentage'], 1)
gen_merge_3 = gen_merge_3[['Gender','Purchase_Count', 'Avg_Purchase_Price', 'Total_Purchase_Value', 'Avg Tot per Person by Gen']]
gen_merge_3.set_index('Gender', inplace=True)


gen_merge_3

,Purchase_Count,Avg_Purchase_Price,Total_Purchase_Value,Avg Tot per Person by Gen
Gender,,,,
Male,652,$3.02,"$1,967.64",$4.07
Female,113,$3.20,$361.94,$4.47
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [62]:

# Create bins in which to place values based upon Age
bins = [0,9, 14, 19, 24, 29, 34,39,100]
# Create labels for these bins
group_labels = ["<10", "10-14", "15-19", "20-24", "25-29","30-34","35-39","40+"]
pd.cut(df["Age"], bins, labels=group_labels).head(10)

# Place the data series into a new column inside of the DataFrame
df["Age Group"] = pd.cut(df["Age"], bins, labels=group_labels)

df.head()

# Create Table summarizing users by age group
purch_count_by_age_0=df[['Age Group','Age','SN']].copy()
purch_count_by_age_01=purch_count_by_age_0.groupby('Age Group')['SN'].count().reset_index()
purch_count_by_age_01.rename(columns={purch_count_by_age_01.columns[1]: "Purchase Count"}, inplace=True)
purch_count_by_age_0.drop_duplicates(['SN'],keep='first', inplace=True)


purch_count_by_age=purch_count_by_age_0[['Age Group','SN']].copy()
purch_count_by_age=purch_count_by_age_0.groupby('Age Group')['SN'].count().reset_index()
purch_count_by_age.rename(columns={purch_count_by_age.columns[1]: "User Count"}, inplace=True)
# Set "Age Group" as index
purch_count_by_age.set_index('Age Group', inplace=True)


# Store values for purchases by age group into variables by Age Group

ten = purch_count_by_age.loc["<10", "User Count"]
fourteen= purch_count_by_age.loc["10-14", "User Count"]
nineteen = purch_count_by_age.loc["15-19", "User Count"]
twentyfour = purch_count_by_age.loc["20-24", "User Count"]
twentynine = purch_count_by_age.loc["25-29", "User Count"]
thirtyfour = purch_count_by_age.loc["30-34", "User Count"]
thirtynine = purch_count_by_age.loc["35-39", "User Count"]
fortyplus = purch_count_by_age.loc["40+", "User Count"]



# Calculate Percentages of Total Purchases for each age group
ten_per= (('{0:.2f}%'.format((ten/unique_len)*100)))
fourteen_per = ('{0:.2f}%'.format((fourteen/unique_len)*100))
nineteen_per = ('{0:.2f}%'.format((nineteen/unique_len)*100))
twentyfour_per = ('{0:.2f}%'.format((twentyfour/unique_len)*100))
twentynine_per = ('{0:.2f}%'.format((twentynine/unique_len)*100))
thirtyfour_per = ('{0:.2f}%'.format((thirtyfour/unique_len)*100))
thirtynine_per = ('{0:.2f}%'.format((thirtynine/unique_len)*100))
fortyplus_per = ('{0:.2f}%'.format((fortyplus/unique_len)*100))

# .transpose function makes it vetical not horizontal
purch_count_by_age
ages_df=pd.DataFrame({
                        "<10":[ten_per],
                        "10-14":[fourteen_per],
                        "15-19":[nineteen_per],
                        "20-24":[twentyfour_per],
                        "25-29":[twentynine_per],
                        "30-34":[thirtyfour_per],
                        "35-39":[thirtynine_per],
                        "40+":[fortyplus_per] }).transpose()

#Rename Index
ages_df.index.name = ('Age Group')
ages_df.rename(columns={ages_df.columns[0]: "% of Total"}, inplace=True)
ages_df

# Create Merge Table
age_merge= pd.merge(purch_count_by_age, ages_df, on="Age Group")
age_merge

,User Count,% of Total
Age Group,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [63]:
# Define new data frame based on age group and price
purch_val_by_age=df.groupby('Age Group')['Price'].sum().reset_index()
# Rename column to revenue from price
purch_val_by_age.rename(columns={purch_val_by_age.columns[1]: "Total_Purchase_Value"}, inplace=True)

# Convert Revenue from object to float
purch_val_by_age['Total_Purchase_Value'] = purch_val_by_age.Total_Purchase_Value.astype(float)

# Merge the two tables to add Revenue by Age group
age_merge_2=pd.merge(age_merge, purch_val_by_age, on = 'Age Group')
#Show Data Types to confirm revenue was changed from object to float
#age_merge_2.dtypes

age_merge_2['Avg_Total_Purchase_Per_Person']= (age_merge_2['Total_Purchase_Value'])/(age_merge_2['User Count'])

# Must convert to float before formatting. Error received even when placed after and in same table
#Format Avg Price
age_merge_2['Avg_Total_Purchase_Per_Person']=age_merge_2['Avg_Total_Purchase_Per_Person'].map("${:,.2f}".format) 
age_merge_2.set_index('Age Group', inplace=True)
#top_spenders_3 = top_spenders_3[['SN','Purchase Count', 'Avg Purchase Price', 'Purchase Value']]
age_merge_3=pd.merge(purch_count_by_age_01,age_merge_2, on='Age Group')
age_merge_3['Avg Price']=age_merge_3['Total_Purchase_Value'] /age_merge_3['Purchase Count'] 


age_merge_3['Total_Purchase_Value'] = age_merge_3['Total_Purchase_Value'].map("${:,.2f}".format)
age_merge_3['Avg Price'] = age_merge_3['Avg Price'].map("${:,.2f}".format)
age_merge_3.set_index('Age Group', inplace=True)
age_merge_3 = age_merge_3[['Purchase Count', 'Avg Price','Total_Purchase_Value','Avg_Total_Purchase_Per_Person']]




#Display Summary Table
age_merge_3

,Purchase Count,Avg Price,Total_Purchase_Value,Avg_Total_Purchase_Per_Person
Age Group,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [64]:
top_spenders=df.groupby('SN')['Price'].sum().reset_index()
top_spenders_2=df.groupby('SN')['Purchase ID'].count().reset_index()
top_spenders_2.rename(columns={top_spenders_2.columns[1]: "Purchase Count"}, inplace=True)
top_spenders_2=top_spenders_2.sort_values(['Purchase Count'], ascending=[False])

top_spenders.set_index('SN', inplace=True)
top_spenders=top_spenders.sort_values(['Price'], ascending=[False])
top_spenders.rename(columns={top_spenders.columns[0]: "Purchase Value"}, inplace=True)
#top_spenders['Purchase Value']=top_spenders['Purchase Value'].map("${:,.2f}".format) 
top_spenders_3=pd.merge(top_spenders,top_spenders_2,on='SN')
top_spenders_3['Avg Purchase Price']= (top_spenders_3['Purchase Value']/top_spenders_3['Purchase Count'])
top_spenders_3['Purchase Value']=top_spenders_3['Purchase Value'].map("${:,.2f}".format) 
top_spenders_3['Avg Purchase Price']=top_spenders_3['Avg Purchase Price'].map("${:,.2f}".format) 
top_spenders_3 = top_spenders_3[['SN','Purchase Count', 'Avg Purchase Price', 'Purchase Value']]
top_spenders_3.set_index('SN', inplace=True)

top_spenders_3.head()

,Purchase Count,Avg Purchase Price,Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [65]:
item_id=df['Item ID']
item_name = df['Item Name']
price = df['Price']

popular=df[['Item ID','Item Name','Price']].copy()
popular_2=popular.groupby(['Item ID','Item Name']).count().reset_index()
popular_2.rename(columns={popular_2.columns[2]: "Total Times Bought"}, inplace=True)

#1 is the axis number (0 for rows and 1 for columns.)
popular_2 = popular_2.drop('Item ID', 1)


popular_3=pd.merge(popular, popular_2, on = 'Item Name')
popular_3['Price'] = popular_3.Price.astype(float)
#popular_3['Price'] = popular_3.Price.astype(int)


tot_val=(popular_3['Price']*popular_3['Total Times Bought']).astype(float)

popular_3['Price']=popular_3['Price'].map("${:,.2f}".format) 
popular_3=popular_3.sort_values(['Total Times Bought', 'Item ID'], ascending=[False, False])

# Only worked when stored in tot_val variable above. Could not create column and multiply the two columns in one line
popular_3['Total_Purchase_Value']=tot_val
popular_3['Total_Purchase_Value']=popular_3['Total_Purchase_Value'].map("${:,.2f}".format) 
#popular_3

# Below Table Drops Duplicates. keep = 'first' says keep one of each repeat row, otherwise every duplicate will be dropped
popular_4=popular_3
popular_4.drop_duplicates(['Item ID'],keep='first', inplace=True)
popular_4 = popular_4[['Item ID', 'Item Name', 'Total Times Bought', 'Price', 'Total_Purchase_Value']]
popular_4.set_index(['Item ID','Item Name'], inplace=True)

popular_4.head()


,,Total Times Bought,Price,Total_Purchase_Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
103,Singed Scalpel,8,$4.35,$34.80


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [66]:
popular_5=popular_3
popular_5['Total_Purchase_Value']=popular_5['Total_Purchase_Value'].replace('\$','',regex=True)
popular_5['Total_Purchase_Value'] = popular_5.Total_Purchase_Value.astype(float)
popular_5=popular_5.sort_values(['Total_Purchase_Value'], ascending=[False])
popular_5['Total_Purchase_Value']=popular_5['Total_Purchase_Value'].map("${:,.2f}".format) 
popular_5.set_index(['Item ID','Item Name'], inplace=True)
popular_5 = popular_5[['Total Times Bought', 'Price', 'Total_Purchase_Value']]





popular_5.head()


,,Total Times Bought,Price,Total_Purchase_Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
